In [1]:
!pip install gensim
!pip install pyrsm --user

Error processing line 1 of C:\Users\test\AppData\Roaming\Python\Python39\site-packages\matplotlib-3.6.3-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\test\anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of C:\Users\test\AppData\Roaming\Python\Python39\site-packages\matplotlib-3.6.3-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\test\anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [2]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
from collections import defaultdict
from math import log
from gensim.models import TfidfModel
from sklearn.metrics import f1_score
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp

# Functions

In [3]:
# A function used to build a vocabulary based on descending word frequencies 
def build_vocab(sentences):
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return word_counts, vocabulary, vocabulary_inv

In [4]:
# A function used to learn word embeddings through Word2vec module
def get_embeddings(inp_data, vocabulary_inv, size_features=100,
                   mode='skipgram',
                   min_word_count=2,
                   context=5):
    model_name = "embedding"
    model_name = os.path.join(model_name)
    num_workers = 15  # Number of threads to run in parallel
    downsampling = 1e-3  # Downsample setting for frequent words
    print('Training Word2Vec model...')
    # use inp_data and vocabulary_inv to reconstruct sentences
    sentences = [[vocabulary_inv[w] for w in s] for s in inp_data]
    if mode == 'skipgram':
        sg = 1
        print('Model: skip-gram')
    elif mode == 'cbow':
        sg = 0
        print('Model: CBOW')
    embedding_model = word2vec.Word2Vec(sentences, workers=num_workers,
                                        sg=sg,
                                        vector_size=size_features,
                                        min_count=min_word_count,
                                        window=context,
                                        sample=downsampling)
    print("Saving Word2Vec model {}".format(model_name))
    embedding_weights = np.zeros((len(vocabulary_inv), size_features))
    for i in range(len(vocabulary_inv)):
        word = vocabulary_inv[i]
        if word in embedding_model.wv:
            embedding_weights[i] = embedding_model.wv[word]
        else:
            embedding_weights[i] = np.random.uniform(-0.25, 0.25,
                                                     embedding_model.vector_size)
    return embedding_weights, embedding_model

In [5]:
def preprocess_df(df):
    # get English stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')
    stop_words.add('The')
    # prepare translation table to translate punctuation to space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    preprocessed_sentences = []
    for i, row in df.iterrows():
        sent = row["text"]
        sent_nopuncts = sent.translate(translator)
        words_list = sent_nopuncts.strip().split()
        filtered_words = [word for word in words_list if word not in stop_words and len(word) != 1] # also skip space from above translation
        preprocessed_sentences.append(" ".join(filtered_words))
    df["text"] = preprocessed_sentences
    return df

In [6]:
def process_attributes(df, test=False):
    attribute_list = ['name','stars','review_count','attributes.OutdoorSeating','attributes.BusinessAcceptsCreditCards','attributes.RestaurantsTableService',
    'attributes.RestaurantsReservations','attributes.RestaurantsPriceRange2','attributes.HasTV','attributes.RestaurantsGoodForGroups','attributes.Caters',
    'attributes.RestaurantsTakeOut','attributes.RestaurantsDelivery','attributes.GoodForKids', 'text']
    if test == False:
        attribute_list.append('label')
    else:
        pass
    df = df[attribute_list]
    col_names = df.columns
    col_name_clean = list(df[attribute_list].columns.str.replace('attributes.', ''))
    new_column_names = {column: column.replace('attributes.', '') for column in col_names}
    df = df.rename(columns=new_column_names)
    d = {'False': 0, 'True': 1}
    #clean b'
    for col in df:
        if col != 'text' and col != 'label':
            try:
                df[col] = df[col].str.extract(r"b'(.*?)'")
                df[col] = df[col].map(d)
            except:
                pass
    df = df.fillna(0)

    df[['OutdoorSeating','BusinessAcceptsCreditCards','RestaurantsTableService','RestaurantsReservations','RestaurantsPriceRange2','HasTV','RestaurantsGoodForGroups',
    'Caters','RestaurantsTakeOut','RestaurantsDelivery','GoodForKids']] = df[['OutdoorSeating','BusinessAcceptsCreditCards','RestaurantsTableService',
    'RestaurantsReservations','RestaurantsPriceRange2','HasTV','RestaurantsGoodForGroups','Caters','RestaurantsTakeOut','RestaurantsDelivery','GoodForKids']].astype('category')

    df[['OutdoorSeating','BusinessAcceptsCreditCards','RestaurantsTableService','RestaurantsReservations','RestaurantsPriceRange2','HasTV','RestaurantsGoodForGroups',
    'Caters','RestaurantsTakeOut','RestaurantsDelivery','GoodForKids']] = df[['OutdoorSeating','BusinessAcceptsCreditCards','RestaurantsTableService',
    'RestaurantsReservations','RestaurantsPriceRange2','HasTV','RestaurantsGoodForGroups','Caters','RestaurantsTakeOut','RestaurantsDelivery','GoodForKids']].astype('category')

    return df

In [7]:
def get_train_vali_test(X, y, ratio_train = 0.8, ratio_val = 0.1, ratio_test = 0.1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=42, shuffle=True)
    ratio_remaining = 1 - ratio_test
    ratio_val_adjusted = ratio_val / ratio_remaining
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=ratio_val_adjusted, random_state=42, shuffle=True)
    return X_train, X_test, X_val, y_train, y_test, y_val

# Main

In [8]:
data_path = "/."

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train["text"] = df_train["review"]
df_test["text"] = df_test["review"]
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)
df_train = process_attributes(df_train)
df_test = process_attributes(df_test, test=True)

C:\Users\test\AppData\Local\Temp\ipykernel_2276\2382970569.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  col_name_clean = list(df[attribute_list].columns.str.replace('attributes.', ''))


In [9]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=True,
                        preprocessor=None,  # applied preprocessor in Data Cleaning
                        tokenizer=word_tokenize,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True,
                        stop_words= 'english',
                        max_df=0.5,
                        sublinear_tf=True)

In [10]:
X =  tfidf.fit_transform(df_train["text"])

In [11]:
X_train, X_test, X_val, y_train, y_test, y_val = get_train_vali_test(X, df_train['label'])

In [12]:
#tfidf_vec =  tfidf.fit_transform(X_train["text"])
#additional_features = sp.csr_matrix(X_train.drop(['text'], axis=1))
#X_train = sp.hstack((tfidf_vec,additional_features))

# Model Training

## logit

In [41]:
clf = LogisticRegression(max_iter=100000000, C=10, class_weight='balanced').fit(X_train, y_train)
f1_score(y_test, clf.predict(X_test), average='micro')

0.8053231939163498

In [42]:
f1_score(y_val, clf.predict(X_val), average='micro')

0.8000000000000002

In [15]:
df_X_test = tfidf.transform(df_test['text'])

In [39]:
param_grid_lr = {"C": [100, 10, 1.0, 0.1, 0.01], "class_weight": ['balanced', None]}

clf_lr_cv = GridSearchCV(clf,
                    param_grid_lr,
                    scoring = "f1_micro",
                    cv = 5,
                    n_jobs = 4,
                    verbose = 10)
#clf_lr_cv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5,
             estimator=LogisticRegression(C=10, class_weight='balanced',
                                          max_iter=100000000),
             n_jobs=4,
             param_grid={'C': [100, 10, 1.0, 0.1, 0.01],
                         'class_weight': ['balanced', None]},
             scoring='f1_micro', verbose=10)

In [43]:
clf_lr_cv.best_params_

{'C': 1.0, 'class_weight': 'balanced'}

## RF

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf_rf = RandomForestClassifier(max_depth=10, random_state=0, class_weight='balanced')
clf_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, random_state=0)

In [26]:
param_grid_rf = {"max_depth": [100, 200], "n_estimators": [100], "max_features":['sqrt'],\
                 "class_weight": ['balanced', None]}
                 

In [27]:
clf_rf_cv = GridSearchCV(clf_rf,
                    param_grid_rf,
                    scoring = "f1_micro",
                    cv = 5,
                    n_jobs = 4,
                    verbose = 10)

In [28]:
clf_rf_cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              max_depth=10, random_state=0),
             n_jobs=4,
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': [100, 200], 'max_features': ['sqrt'],
                         'n_estimators': [100]},
             scoring='f1_micro', verbose=10)

In [44]:
# in your implemetation, create the output file using the same format
dic = {"Id": [], "Predicted": []}
df_X_test = tfidf.transform(df_test['text'])
preds = clf.predict(df_X_test)
for i, pred in enumerate(preds):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)

dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv("predicted.csv", index=False)